In [16]:
%matplotlib inline
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:75% !important; }</style>"))

import sys
sys.dont_write_bytecode = True

import numpy as np, cv2
from lxml import etree
from PIL import Image, ImageDraw
import os, json
from collections import OrderedDict
import matplotlib.pyplot as plt

In [17]:
clevr_colors = ["red", "blue", "green", "brown", "purple", "cyan", "yellow"]

color_mean_dict = {"red":[140, 64, 61], "yellow":[135, 127, 52],"brown":[112, 84, 46],
                   "cyan":[54, 126, 118], "green":[50, 96, 42],"blue":[50, 70, 120],
                   "purple":[95, 52, 113]}
color_thres_dict = {"red":5, "yellow":5, "brown":5,
                   "cyan":5, "green":5, "blue":5,
                    "purple":5}

In [18]:
def color_range(color_string):
    [red, green, blue] = color_mean_dict[color_string]
    thres = color_thres_dict[color_string]
    
    color = np.uint8([[[blue, green, red]]])
    hsv_color = cv2.cvtColor(color, cv2.COLOR_BGR2HSV)
    hue = hsv_color[0][0][0]
    
    low = max(0, hue - thres)
    high = min(360, hue + thres)
    lower_range = np.array([low, 40, 40], dtype=np.uint8) #very low range for dealing with metallics
    upper_range = np.array([high, 255, 255], dtype=np.uint8)
    
    return (lower_range, upper_range)

# -----------------------------------------------------------------------

color_filter_dict = {x: color_range(x) for x in clevr_colors}

In [19]:
def imshow(img):
    plt.imshow(img[...,::-1])
    plt.pause(0.04)
    return

In [20]:
img_dir = "images/train"
scene_dir = "scenes/train"
img_names = sorted(os.listdir(img_dir))


In [21]:
def load_json(json_path):
    with open(json_path, 'r') as f:
        scene = json.load(f, object_pairs_hook=OrderedDict)
    return scene

In [22]:
def get_bb(img_name, color, pixel_coords, default_size=20, padding=5):
    img = cv2.imread(os.path.join(img_dir, img_name))
    hsv_img = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    
    (low, high) = color_filter_dict[color]
    mask = cv2.inRange(hsv_img, low, high)
    mask = cv2.medianBlur(mask,11)

    (_, cnts, hierarchy) = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    #     ------Fill all holes---------------
    for i in range(len(cnts)):
        cv2.drawContours(mask, cnts, i, (255,255,255), cv2.FILLED)

    # ---------------------------------------
    (_, cnts, hierarchy) = cv2.findContours(mask, cv2.RETR_CCOMP, cv2.CHAIN_APPROX_SIMPLE)

    if(len(cnts) == 0):
        xmin = pixel_coords[0] - default_size; ymin = pixel_coords[1] - default_size
        xmax = pixel_coords[0] + default_size; ymax = pixel_coords[1] + default_size
    elif(cv2.contourArea(cnts[0]) < 20):
        xmin = pixel_coords[0] - default_size; ymin = pixel_coords[1] - default_size
        xmax = pixel_coords[0] + default_size; ymax = pixel_coords[1] + default_size
    else:
        rect = cv2.boundingRect(cnts[0])
        xmin,ymin,w,h = rect
        xmax = xmin + w; ymax = ymin + h
    
    xmin = xmin - padding; ymin = ymin - padding
    xmax = xmax + padding; ymax = ymax + padding
    
    height, width, channels = img.shape
    xmin = max(0, xmin); ymin = max(0, ymin)
    xmax = min(width, xmax); ymax = min(height, ymax)
    
#     cv2.rectangle(img, (xmin, ymin), (xmax, ymax), (255,0,0), 2)
#     imshow(img)

    return (xmin, ymin, xmax, ymax)

In [23]:
def get_annotations(img_name):
    scene = load_json(os.path.join(scene_dir, img_name[:-4] + ".json"))
    object_info = scene["objects"][0]
    size = object_info["size"]
    shape = object_info["shape"]
    material = object_info["material"]
    color = object_info["color"]
#     x = object_info["3d_coords"][0]
#     y = object_info["3d_coords"][1]
#     theta = object_info["rotation"]
#     bb is (xmin, ymin, xmax, ymax)
    bb = get_bb(img_name, color, object_info["pixel_coords"])

    dict = {"size": size, "shape": shape, "material": material, "bb":bb}
    return dict
    

In [47]:
def write_xml(img_name):
    dict = get_annotations(img_name)
    
    img = Image.open(os.path.join(img_dir, img_name))
    bg = Image.new("RGB", img.size, (255,255,255))
    bg.paste(img,img)
    
    img_name = img_name[:-4]+".JPEG" #update to jpg
    
    bg.save(r"JPEGImages/"+img_name)
    width, height = img.size
    
    
    root = etree.Element("annotation")
    etree.SubElement(root, "folder").text = "clevr_single"
    etree.SubElement(root, "filename").text = img_name

    source = etree.SubElement(root, "source")
    etree.SubElement(source, "database").text = "CLEVR_SINGLE"

    size = etree.SubElement(root, "size")
    etree.SubElement(size, "width").text = str(width)
    etree.SubElement(size, "height").text = str(height)
    etree.SubElement(size, "depth").text = str(3)

    etree.SubElement(root, "segmented").text = str(0)

    object = etree.SubElement(root, "object")
    etree.SubElement(object, "name").text = str(dict["shape"])
    etree.SubElement(object, "pose").text = "unspecified"
    etree.SubElement(object, "truncated").text = str(0)
    etree.SubElement(object, "difficult").text = str(0)
    bndbox = etree.SubElement(object, "bndbox")
    etree.SubElement(bndbox, "xmin").text = str(dict["bb"][0])
    etree.SubElement(bndbox, "ymin").text = str(dict["bb"][1])
    etree.SubElement(bndbox, "xmax").text = str(dict["bb"][2])
    etree.SubElement(bndbox, "ymax").text = str(dict["bb"][3])

    tree = etree.ElementTree(root)
    tree.write("Annotations/"+img_name[:-5]+".xml",pretty_print=True )
    
    return

In [48]:
img_dir = "images/train"
img_names = sorted(os.listdir(img_dir))


In [49]:
len(img_names)

6000

In [50]:
write_xml(img_names[0])

In [ ]:
for i in range(6000):
    print(img_names[i])
    write_xml(img_names[i])

CLEVR_train_000000.png
CLEVR_train_000001.png
CLEVR_train_000002.png
CLEVR_train_000003.png
CLEVR_train_000004.png
CLEVR_train_000005.png
CLEVR_train_000006.png
CLEVR_train_000007.png
CLEVR_train_000008.png
CLEVR_train_000009.png
CLEVR_train_000010.png
CLEVR_train_000011.png
CLEVR_train_000012.png
CLEVR_train_000013.png
CLEVR_train_000014.png
CLEVR_train_000015.png
CLEVR_train_000016.png
CLEVR_train_000017.png
CLEVR_train_000018.png
CLEVR_train_000019.png
CLEVR_train_000020.png
CLEVR_train_000021.png
CLEVR_train_000022.png
CLEVR_train_000023.png
CLEVR_train_000024.png
CLEVR_train_000025.png
CLEVR_train_000026.png
CLEVR_train_000027.png
CLEVR_train_000028.png
CLEVR_train_000029.png
CLEVR_train_000030.png
CLEVR_train_000031.png
CLEVR_train_000032.png
CLEVR_train_000033.png
CLEVR_train_000034.png
CLEVR_train_000035.png
CLEVR_train_000036.png
CLEVR_train_000037.png
CLEVR_train_000038.png
CLEVR_train_000039.png
CLEVR_train_000040.png
CLEVR_train_000041.png
CLEVR_train_000042.png
CLEVR_train

CLEVR_train_000361.png
CLEVR_train_000362.png
CLEVR_train_000363.png
CLEVR_train_000364.png
CLEVR_train_000365.png
CLEVR_train_000366.png
CLEVR_train_000367.png
CLEVR_train_000368.png
CLEVR_train_000369.png
CLEVR_train_000370.png
CLEVR_train_000371.png
CLEVR_train_000372.png
CLEVR_train_000373.png
CLEVR_train_000374.png
CLEVR_train_000375.png
CLEVR_train_000376.png
CLEVR_train_000377.png
CLEVR_train_000378.png
CLEVR_train_000379.png
CLEVR_train_000380.png
CLEVR_train_000381.png
CLEVR_train_000382.png
CLEVR_train_000383.png
CLEVR_train_000384.png
CLEVR_train_000385.png
CLEVR_train_000386.png
CLEVR_train_000387.png
CLEVR_train_000388.png
CLEVR_train_000389.png
CLEVR_train_000390.png
CLEVR_train_000391.png
CLEVR_train_000392.png
CLEVR_train_000393.png
CLEVR_train_000394.png
CLEVR_train_000395.png
CLEVR_train_000396.png
CLEVR_train_000397.png
CLEVR_train_000398.png
CLEVR_train_000399.png
CLEVR_train_000400.png
CLEVR_train_000401.png
CLEVR_train_000402.png
CLEVR_train_000403.png
CLEVR_train